In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall nltk -y
!pip install texthero

Uninstalling nltk-3.2.5:
  Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 245kB 6.1MB/s 
     |████████████████████████████████| 1.4MB 8.9MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434675 sha256=f04665254f6aa463c9ceac1fecfc10d191733056f375de50c248726580ce307b
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk


In [3]:
import os

import numpy as np
import pandas as pd

import dill
from sklearn.feature_extraction.text import TfidfVectorizer
import gc
from tqdm import tqdm
import time

import gensim
from gensim.models import Word2Vec
from gensim.models import FastText
import hashlib
import nltk

import texthero as hero
from texthero import preprocessing

INPUT_DIR = '/content/drive/MyDrive/citation_prediction/input/'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


データの読み込み

In [4]:
all_title_abstract = pd.read_feather(os.path.join(INPUT_DIR, 'all_title_abstract_df.feather'))
all_title_abstract.head()

,title,abstract,cites,clean_title,clean_abstract
0,Heavy-Flavour Production at HERA,We review the theoretical and experimental s...,NaN,heavy flavour production hera,review theoretical experimental status hea...
1,Spectropolarimetric Constraints on the Nature ...,While it is well recognized that interstella...,7.0,spectropolarimetric constraints nature inte...,well recognized interstellar grains made ...
2,A joint analysis of Planck and BICEP2 B modes ...,We analyze BICEP2 and Planck data using a mo...,188.0,joint analysis planck bicep2 b modes includ...,analyze bicep2 planck data using model inc...
3,Molecular movie of ultrafast coherent rotation...,Recording molecular movies on ultrafast time...,8.0,molecular movie ultrafast coherent rotational...,recording molecular movies ultrafast timescal...
4,A Modified Mixed Domain Method for Modeling Ac...,"In this paper, phase correction and amplitud...",NaN,modified mixed domain method modeling acoust...,paper phase correction amplitude compensati...


In [5]:
start = time.time()

sentences = []
print ("Parsing sentences from training set...")

# Loop over each news article.
for review in tqdm(all_title_abstract['clean_abstract']):
    try:
        # Split a review into parsed sentences.
        result = review
        h = result.split(" ")
        h = list(filter(("").__ne__, h))
        sentences.append(h)
    except:
        continue

num_features = 200     # Word vector dimensionality
min_word_count = 20   # Minimum word count
num_workers = 1       # Number of threads to run in parallel
context = 10           # Context window size
downsampling = 1e-3   # Downsample setting for frequent words
seed = 42

hashfxn = lambda x: int(hashlib.md5(str(x).encode()).hexdigest(), 16)

print ("Training FastText model...")
# Train FastText model.
model = FastText(sentences, workers=num_workers, hs = 0, sg = 1, negative = 10, iter = 5,\
            size=num_features, min_count = min_word_count, hashfxn=hashfxn,\
            window = context, sample = downsampling, seed=seed)

model_name = str(num_features) + "features_" + str(min_word_count) + "minwords_" \
+ str(context) + "context_len2alldata"
model.init_sims(replace=True)

endmodeltime = time.time()

print ("time : ", endmodeltime-start)

  0%|          | 4308/910608 [00:00<00:21, 43077.68it/s]

Parsing sentences from training set...


100%|██████████| 910608/910608 [00:26<00:00, 34393.11it/s]


Training FastText model...
time :  24401.888252973557


In [6]:
tqdm.pandas()

def des_to_mean_vec(text):
    
    try:
        text = text.strip()
        text_ls = [s for s in text.split(' ') if '' != s]
        return np.mean([model.wv[word] for word in text_ls if word in model.wv.index2word], axis=0)
    except:
        return np.nan

df = all_title_abstract['clean_abstract'].progress_apply(lambda x: des_to_mean_vec(x))
df.head()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version

  4%|▎         | 31974/910608 [07:24<3:13:17, 75.76it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

100%|██████████| 910608/910608 [3:50:18<00:00, 65.90it/s]


0    [0.05084451, -0.09144361, 0.007336243, 0.00654...
1    [0.08030317, -0.06594277, -0.008051708, -0.028...
2    [0.041681033, -0.06944177, 0.013492349, -0.010...
3    [0.0299834, -0.07853553, 0.0026571655, -0.0128...
4    [0.045683786, -0.051367216, 0.008049393, -0.02...
Name: clean_abstract, dtype: object

In [7]:
abstract_df = df.apply(pd.Series)
abstract_df.columns = ['fasttext_abstract_' + str(col) for col in abstract_df.columns]
abstract_df.head()

,fasttext_abstract_0,fasttext_abstract_1,fasttext_abstract_2,fasttext_abstract_3,fasttext_abstract_4,fasttext_abstract_5,fasttext_abstract_6,fasttext_abstract_7,fasttext_abstract_8,fasttext_abstract_9,fasttext_abstract_10,fasttext_abstract_11,fasttext_abstract_12,fasttext_abstract_13,fasttext_abstract_14,fasttext_abstract_15,fasttext_abstract_16,fasttext_abstract_17,fasttext_abstract_18,fasttext_abstract_19,fasttext_abstract_20,fasttext_abstract_21,fasttext_abstract_22,fasttext_abstract_23,fasttext_abstract_24,fasttext_abstract_25,fasttext_abstract_26,fasttext_abstract_27,fasttext_abstract_28,fasttext_abstract_29,fasttext_abstract_30,fasttext_abstract_31,fasttext_abstract_32,fasttext_abstract_33,fasttext_abstract_34,fasttext_abstract_35,fasttext_abstract_36,fasttext_abstract_37,fasttext_abstract_38,fasttext_abstract_39,...,fasttext_abstract_160,fasttext_abstract_161,fasttext_abstract_162,fasttext_abstract_163,fasttext_abstract_164,fasttext_abstract_165,fasttext_abstract_166,fasttext_abstract_167,fasttext_abstract_168,fasttext_abstract_169,fasttext_abstract_170,fasttext_abstract_171,fasttext_abstract_172,fasttext_abstract_173,fasttext_abstract_174,fasttext_abstract_175,fasttext_abstract_176,fasttext_abstract_177,fasttext_abstract_178,fasttext_abstract_179,fasttext_abstract_180,fasttext_abstract_181,fasttext_abstract_182,fasttext_abstract_183,fasttext_abstract_184,fasttext_abstract_185,fasttext_abstract_186,fasttext_abstract_187,fasttext_abstract_188,fasttext_abstract_189,fasttext_abstract_190,fasttext_abstract_191,fasttext_abstract_192,fasttext_abstract_193,fasttext_abstract_194,fasttext_abstract_195,fasttext_abstract_196,fasttext_abstract_197,fasttext_abstract_198,fasttext_abstract_199
0,0.050845,-0.091444,0.007336,0.006546,-0.017894,-0.044426,-0.026023,0.102765,0.021151,-0.019797,-0.010982,0.017517,0.012306,-0.045618,0.078920,-0.053767,-0.016750,0.066131,-0.057374,0.040446,0.005487,-0.042632,0.021723,-0.000570,-0.077572,0.044867,-0.031104,-0.011891,0.048771,-0.020382,0.032146,-0.040153,0.094089,-0.000257,0.026084,-0.022011,-0.044312,0.026178,-0.035891,-0.085723,...,-0.007394,-0.005429,0.044931,0.024530,-0.099204,-0.055927,-0.003341,0.046970,-0.059572,0.017766,-0.009478,-0.006686,-0.042427,0.062113,0.041980,0.023925,-0.014659,-0.014482,0.009528,-0.094117,0.001793,0.045220,-0.076598,0.013617,0.068791,-0.060148,0.006597,0.035036,0.011890,0.056463,0.006744,-0.075465,-0.044763,0.001359,-0.043083,0.005208,0.002111,0.007594,-0.000028,0.064086
1,0.080303,-0.065943,-0.008052,-0.028439,0.035818,-0.008764,-0.018645,0.045126,0.005049,-0.050975,0.034583,-0.030788,-0.040805,-0.015986,0.042775,-0.027504,0.055815,0.025603,0.015123,0.028701,0.014559,-0.033917,-0.015217,-0.023733,-0.041885,0.015605,-0.046858,0.007950,0.012027,-0.037455,0.064867,-0.039659,-0.016365,0.022224,-0.011403,0.032937,-0.003295,0.021527,-0.015703,0.020253,...,0.013768,0.027186,0.054613,-0.018038,-0.022565,-0.071782,-0.049431,0.030357,-0.013771,0.061911,-0.028589,-0.043037,-0.022701,0.000932,0.000009,-0.022508,-0.078257,-0.042218,0.071240,-0.012131,0.026460,-0.013632,-0.065061,0.032810,0.024275,0.028147,-0.036229,-0.012566,0.063694,0.027030,0.056265,-0.020993,-0.036400,-0.001671,-0.009431,-0.031474,-0.029356,-0.014566,-0.008456,0.051272
2,0.041681,-0.069442,0.013492,-0.010745,0.031950,-0.010009,-0.037131,0.040752,0.008268,-0.041449,0.005334,-0.051313,-0.050743,-0.019319,0.006686,-0.053932,0.051783,0.020832,0.006660,-0.014202,0.009913,-0.033610,0.004130,-0.031106,-0.083360,0.005864,-0.039000,0.021731,0.015516,-0.020217,0.065487,-0.022043,0.015431,0.038003,0.013725,0.009968,-0.022034,0.011860,-0.051184,-0.008681,...,0.002964,-0.008870,0.076686,-0.013843,-0.015300,-0.060729,-0.030031,0.048949,-0.033164,0.026531,-0.007242,-0.012161,-0.025995,0.014518,0.004307,0.038368,-0.042085,-0.017649,0.065910,-0.026920,0.010791,-0.012712,-0.064307,0.004192,-0.004828,0.004569,-0.006643,-0.017516,0.055169,-0.027172,0.048197,-0.031967,-0.011874,0.006852,0.012295,-0.015849,0.021023,-0.012941

In [8]:
abstract_df.to_feather(os.path.join(INPUT_DIR, 'abstract_fasttext_df.feather'))

In [9]:
file = os.path.join(INPUT_DIR, 'fasttext_model_abstract.dill')
dill.dump(model, open(file,'wb'))